In [2]:
from flask import Flask, request, jsonify
from threading import Thread
import json
from datetime import datetime, timedelta
import os
import requests
import re

from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build


import pytz
from dateutil import parser as dateparser

TIMEZONE_ABBREVIATIONS = {
    "IST": "Asia/Kolkata",
    "EST": "US/Eastern",
    "EDT": "US/Eastern",
    "PST": "US/Pacific",
    "PDT": "US/Pacific",
    "CST": "US/Central",
    "CDT": "US/Central",
    "MST": "US/Mountain",
    "MDT": "US/Mountain",
    "GMT": "Etc/GMT",
    "UTC": "UTC"
}

def extract_timezone_hint(text):
    for abbr in TIMEZONE_ABBREVIATIONS:
        if re.search(r'\b' + re.escape(abbr) + r'\b', text.upper()):
            return TIMEZONE_ABBREVIATIONS[abbr]
    return None


# ---------- CONFIG ----------
LLM_BASE_URL = "http://localhost:4000/v1/chat/completions"
LLM_MODEL_PATH = "/home/user/Models/meta-llama/Meta-Llama-3.1-8B-Instruct"

# ---------- LLM UTILITIES ----------
def run_llm(messages, max_tokens=256):
    payload = {
        "model": LLM_MODEL_PATH,
        "messages": messages,
        "stream": False,
        "max_tokens": max_tokens
    }
    response = requests.post(LLM_BASE_URL, headers={"Content-Type": "application/json"}, json=payload, timeout=45)
    content = response.json()["choices"][0]["message"]["content"]
    try:
        return json.loads(content)
    except Exception:
        return content

def improved_llm_prompt(email_text, email_subject=None):
    subject_part = f'\nThe email subject is: "{email_subject}". Use this as meeting_subject unless the email body gives a better one.' if email_subject else ""
    messages = [
        {"role": "system", "content": (
            "You are a meeting scheduling agent. Extract ONLY the following as a valid JSON object:\n"
            "\"participants\": list of ALL relevant email addresses (include the sender and every invitee mentioned in the email or header, append @amd.com if only a name; never use 'you', always use emails),\n"
            "\"meeting_duration\": integer (minutes, fallback to 30 if not present),\n"
            "\"time_constraints\": text string (the exact/fuzzy meeting time),\n"
            "\"meeting_subject\": subject (use input subject if present, otherwise infer from body).\n"
            "Reply with ONLY the JSON object, no markdown or text."
            + subject_part
        )},
        {"role": "user", "content": email_text}
    ]
    return run_llm(messages)

def polite_explanation_prompt(original_time, suggested_time, attendees, subject):
    messages = [
        {"role": "system", "content": (
            "You are a professional scheduling assistant. If a requested meeting time was not available, "
            "write a concise, friendly explanation (for a calendar event or email) apologizing and clearly stating the new time and reason."
        )},
        {"role": "user", "content": (
            f"The requested meeting '{subject}' with {', '.join(attendees)} could not be scheduled at {original_time}. "
            f"The next best available slot is {suggested_time}. Draft a brief note for the meeting invitation."
        )}
    ]
    return run_llm(messages, max_tokens=100)

# ---------- CALENDAR ACCESS ----------
def retrive_calendar_events(user, start, end):
    events_list = []
    token_path = "../Keys/"+user.split("@")[0]+".token"
    if not os.path.isfile(token_path):
        print(f"WARNING: No calendar token found for {user}. Returning empty event list.")
        return []
    user_creds = Credentials.from_authorized_user_file(token_path)
    calendar_service = build("calendar", "v3", credentials=user_creds)
    events_result = calendar_service.events().list(calendarId='primary', timeMin=start, timeMax=end, singleEvents=True, orderBy='startTime').execute()
    events = events_result.get('items')
    for event in events:
        attendee_list = []
        try:
            for attendee in event.get("attendees", []):
                attendee_list.append(attendee['email'])
        except:
            attendee_list.append("SELF")
    
        # ✅ Safe start and end time extraction
        start_time = event["start"].get("dateTime") or event["start"].get("date") + "T00:00:00+05:30"
        end_time = event["end"].get("dateTime") or event["end"].get("date") + "T23:59:00+05:30"
    
        events_list.append({
            "StartTime": start_time,
            "EndTime": end_time,
            "NumAttendees": len(set(attendee_list)),
            "Attendees": list(set(attendee_list)),
            "Summary": event.get("summary", "No Title")
        })

    return events_list

# ---------- TIME SLOT & CONFLICT UTILITIES ----------
def iso_to_datetime(iso):
    dt = dateparser.parse(iso)
    if not dt.tzinfo:
        dt = pytz.timezone("Asia/Kolkata").localize(dt)
    return dt
    
def datetime_to_iso(dt):
    if dt.tzinfo is None:
        dt = pytz.timezone("Asia/Kolkata").localize(dt)
    return dt.strftime("%Y-%m-%dT%H:%M:%S%z")
    
def is_user_free(events, req_start, req_end):
    req_start = iso_to_datetime(req_start)
    req_end = iso_to_datetime(req_end)
    for event in events:
        ev_start = iso_to_datetime(event["StartTime"])
        ev_end = iso_to_datetime(event["EndTime"])
        if (ev_start < req_end and ev_end > req_start):
            return False
    return True

def get_free_slots(events, work_start="00:00", work_end="23:59", meeting_date=None):
    day = meeting_date.strftime("%Y-%m-%d")
    tz = pytz.timezone("Asia/Kolkata")
    work_start_dt = tz.localize(datetime.strptime(f"{day}T{work_start}", "%Y-%m-%dT%H:%M"))
    work_end_dt = tz.localize(datetime.strptime(f"{day}T{work_end}", "%Y-%m-%dT%H:%M"))
    busy = []
    for event in events:
        try:
            s = iso_to_datetime(event["StartTime"])
            e = iso_to_datetime(event["EndTime"])
            busy.append((s, e))
        except Exception:
            pass
    busy = sorted(busy)
    free = []
    prev_end = work_start_dt
    for s, e in busy:
        if s >= prev_end:
            free.append((prev_end, s))
        prev_end = max(prev_end, e)
    if prev_end <= work_end_dt:
        free.append((prev_end, work_end_dt))
    return free

def find_common_slot(all_free, duration_mins):
    duration = timedelta(minutes=duration_mins)
    candidate_slots = []
    slots = all_free[0]
    for slot in slots:
        s0, e0 = slot
        for others in all_free[1:]:
            overlap_found = False
            for s1, e1 in others:
                start = max(s0, s1)
                end = min(e0, e1)
                if (end - start) >= duration:
                    overlap_found = True
                    candidate_slots.append((start, start + duration))
                    break
            if not overlap_found:
                break
        else:
            return candidate_slots[0] if candidate_slots else None
    return None
    
def extract_time_components(natural_time_str, reference_datetime=None):
    # Allow reference_datetime to be passed for deterministic unit tests
    ref_dt_str = reference_datetime.strftime("%Y-%m-%dT%H:%M:%S") if reference_datetime else "now"
    messages = [
        {"role": "system", "content": (
            "You are a smart time phrase parser for meeting scheduling.\n\n"
            "Instructions:\n"
            "1. Take in casual input such as 'next Friday at 10 AM', 'this Wednesday', 'tomorrow 3pm', 'after a fortnight', 'in 2 weeks', 'in 10 days', 'ASAP', etc.\n"
            "2. If the phrase specifies a relative duration (like 'after a fortnight', 'in 10 days', 'after 2 weeks', 'after 3 days'), compute the *target date* by adding the duration to today's date (reference: " + ref_dt_str + ").\n"
            "   - Examples: 'after a fortnight' = today + 14 days; 'in 2 weeks' = today + 14 days; 'after 3 days' = today + 3 days.\n"
            "3. If the phrase is 'ASAP', return the soonest possible time today (use current time rounded up to the next 30 min slot), or tomorrow 9:00 if today is already fully booked.\n"
            "4. If no time is given, set 'time' to null (let downstream logic pick a default working hour like 09:00 or next available slot).\n"
            "5. If duration is not given, set 'duration_minutes' to 30 by default.\n"
            "6. Output the *computed absolute date* as 'date': 'YYYY-MM-DD' if known, otherwise null.\n"
            "7. Output all of the following fields in a JSON object:\n"
            "   - reference: string, e.g. 'today', 'tomorrow', 'next', 'this', 'in_X_days', etc. or null\n"
            "   - weekday: string, e.g. 'monday', 'friday', etc. or null\n"
            "   - date: YYYY-MM-DD or null\n"
            "   - time: 'HH:MM' in 24-hour format or null\n"
            "   - duration_minutes: integer (default 30)\n"
            "Return only JSON. No extra explanation."
        )},
        {"role": "user", "content": f"Parse this time phrase: \"{natural_time_str}\""}
    ]
    return run_llm(messages)

# ---------- MAIN AGENTIC SCHEDULER ----------
# Define fuzzy time phrases globally
FUZZY_TIME_MAP = {
    "after lunch": "14:00", "lunch": "13:00",
    "afternoon": "15:00", "morning": "10:00",
    "evening": "17:00", "end of the day": "17:30",
    "before lunch": "11:00", "noon": "12:00",
    "night": "20:00",
}

def your_meeting_assistant(data):
    # Use consistent timezone (IST)
    tz = pytz.timezone("Asia/Kolkata")

    # 1. Parse intent using LLM with subject awareness
    intent = improved_llm_prompt(data["EmailContent"], data.get("Subject"))
    emails = [data["From"]] + [a["email"] for a in data.get("Attendees", [])]
    emails = list(set(emails))

    try:
        participants = intent.get("participants", emails)
        if (not isinstance(participants, list)) or (sorted(participants) != sorted(emails)):
            participants = emails
        meeting_subject = intent.get("meeting_subject")
        if not meeting_subject or len(meeting_subject.strip()) < 3:
            meeting_subject = data.get("Subject", "No Subject")
        meeting_duration = int(intent.get("meeting_duration", 30))
        time_constraints = intent.get("time_constraints", "")
    except Exception:
        participants = emails
        meeting_subject = data.get("Subject", "No Subject")
        meeting_duration = 30
        time_constraints = ""

    from datetime import datetime
    from dateutil import parser as dateparser
    
    tz = pytz.timezone("Asia/Kolkata")
    
    # Force correct parsing of 'DD-MM-YYYY' input
    try:
        now = datetime.strptime(data["Datetime"], "%d-%m-%YT%H:%M:%S")
    except Exception as e:
        print("❌ Could not parse Datetime:", e)
        now = datetime.now()
    
    # Make tz-aware
    now = tz.localize(now)

    tz_hint = extract_timezone_hint(data["EmailContent"]) or "Asia/Kolkata"
    tz = pytz.timezone(tz_hint)
    if not now.tzinfo:
        now = tz.localize(now)

    # Get interpreted time structure from LLM
    time_info = extract_time_components(time_constraints, reference_datetime=now)

    print("🔍 LLM time_info:", time_info)

    reference = str(time_info.get("reference", "") or "").lower()
    weekday = str(time_info.get("weekday", "") or "").lower()
    time_str = time_info.get("time", None)
    duration_from_llm = time_info.get("duration_minutes", 30)

    if duration_from_llm:
        meeting_duration = int(duration_from_llm)

    # 🧠 Try fuzzy mapping if LLM output is fuzzy
    if time_str in FUZZY_TIME_MAP:
        print(f"⏱ Interpreted fuzzy time '{time_str}' as {FUZZY_TIME_MAP[time_str]}")
        time_str = FUZZY_TIME_MAP[time_str]

    if not time_str:
        lc_constraints = time_constraints.lower()
        m = re.search(r"after\s+(\d{1,2})(?::(\d{2}))?\s*(am|pm)?", lc_constraints)
        if m:
            hour = int(m.group(1))
            minute = int(m.group(2)) if m.group(2) else 0
            ampm = m.group(3)
            if ampm:
                if ampm.lower() == "pm" and hour < 12:
                    hour += 12
            time_str = f"{hour:02d}:{minute:02d}"
        else:
            for phrase, mapped_time in FUZZY_TIME_MAP.items():
                if phrase in lc_constraints:
                    print(f"🧠 Matched '{phrase}' in original constraint → {mapped_time}")
                    time_str = mapped_time
                    break

    if not time_str:
        time_str = "09:00"

    meeting_date = now

    if reference == "tomorrow":
        meeting_date = now + timedelta(days=1)
    elif reference == "day after tomorrow":
        meeting_date = now + timedelta(days=2)
    elif reference == "today":
        meeting_date = now
    elif weekday:
        weekdays = {
            "monday": 0, "tuesday": 1, "wednesday": 2,
            "thursday": 3, "friday": 4, "saturday": 5, "sunday": 6
        }
        target_idx = weekdays.get(weekday)
        if target_idx is not None:
            current_idx = now.weekday()
            days_ahead = (target_idx - current_idx + 7) % 7
            if reference == "next" or (reference == "this" and days_ahead == 0 and time_str):
                days_ahead += 7
            meeting_date = now + timedelta(days=days_ahead)

    if not reference and weekday == "" and time_str:
        candidate_dt = tz.localize(datetime.combine(now.date(), datetime.strptime(time_str, "%H:%M").time()))
        if candidate_dt < now:
            meeting_date = now + timedelta(days=1)
        else:
            meeting_date = now

    if time_constraints.strip().lower() == "asap" or (reference == "today" and time_str == "09:00"):
        rounded_now = now + timedelta(minutes=(30 - now.minute % 30))
        time_str = rounded_now.strftime("%H:%M")
        meeting_date = now
        print("📍 ASAP detected — using first available today at", time_str)

    requested_start = tz.localize(datetime.combine(meeting_date.date(), datetime.strptime(time_str, "%H:%M").time()))
    if requested_start < now:
        requested_start += timedelta(days=1)
    requested_end = requested_start + timedelta(minutes=meeting_duration)
    req_start_iso = datetime_to_iso(requested_start)
    req_end_iso = datetime_to_iso(requested_end)

    print(f"📆 Final Scheduled Attempt: {req_start_iso} → {req_end_iso}")

    # ... rest of your code continues unchanged ...


    all_free = True
    for email in participants:
        events = retrive_calendar_events(email, req_start_iso, req_end_iso)
        if not is_user_free(events, req_start_iso, req_end_iso):
            all_free = False
            break

    EventStart = req_start_iso
    EventEnd = req_end_iso
    meta_message = ""
    found_slot = all_free

    search_days = 6
    if not found_slot:
        # 1️⃣ FIRST: Try to find slot later in the same day
        same_day_free = []
        for email in participants:
            day_start = f"{meeting_date.strftime('%Y-%m-%d')}T00:00:00+05:30"
            day_end = f"{meeting_date.strftime('%Y-%m-%d')}T23:59:59+05:30"
            events = retrive_calendar_events(email, day_start, day_end)
            free = get_free_slots(events, meeting_date=meeting_date)
            same_day_free.append(free)

        # Filter to only slots after the requested time
        later_slots = []
        for slots in zip(*same_day_free):
            valid_group_slot = []
            for slot in slots:
                start, end = slot
                if start >= requested_start and (end - start).total_seconds() >= meeting_duration * 60:
                    valid_group_slot.append((start, start + timedelta(minutes=meeting_duration)))
            if len(valid_group_slot) == len(participants):
                later_slots = valid_group_slot
                break

        if later_slots:
            slot_start, slot_end = later_slots[0]
            EventStart = datetime_to_iso(slot_start)
            EventEnd = datetime_to_iso(slot_end)
            meta_message = polite_explanation_prompt(
                original_time=f"{req_start_iso} to {req_end_iso}",
                suggested_time=EventStart,
                attendees=participants,
                subject=meeting_subject
            )
            found_slot = True

    # 2️⃣ THEN: If still not found, check future days
    if not found_slot:
        for delta in range(1, search_days):  # skip today, already checked
            future_date = meeting_date + timedelta(days=delta)
            all_free_slots = []
            for email in participants:
                day_start = f"{future_date.strftime('%Y-%m-%d')}T00:00:00+05:30"
                day_end = f"{future_date.strftime('%Y-%m-%d')}T23:59:59+05:30"
                events = retrive_calendar_events(email, day_start, day_end)
                free = get_free_slots(events, meeting_date=future_date)
                all_free_slots.append(free)
            slot = find_common_slot(all_free_slots, meeting_duration)
            if slot:
                slot_start, slot_end = slot
                EventStart = datetime_to_iso(slot_start)
                EventEnd = datetime_to_iso(slot_end)
                meta_message = polite_explanation_prompt(
                    original_time=f"{req_start_iso} to {req_end_iso}",
                    suggested_time=EventStart,
                    attendees=participants,
                    subject=meeting_subject
                )
                found_slot = True
                break

    # 3️⃣ Fallback default
    if not found_slot:
        future_date = meeting_date + timedelta(days=search_days)
        slot_start = tz.localize(future_date.replace(hour=9, minute=0))
        slot_end = slot_start + timedelta(minutes=meeting_duration) 
        EventStart = datetime_to_iso(slot_start)
        EventEnd = datetime_to_iso(slot_end)
        meta_message = f"No common free slot found for {search_days} days. Auto-booked at {EventStart}."


    output = {
        "Request_id": data["Request_id"],
        "Datetime": data["Datetime"],
        "Location": data.get("Location", ""),
        "From": data["From"],
        "Attendees": [],
        "Subject": meeting_subject,
        "EmailContent": data["EmailContent"],
        "EventStart": EventStart,
        "EventEnd": EventEnd,
        "Duration_mins": str(meeting_duration),
        "MetaData": {"polite_note": meta_message}
        
    }
    for email in participants:
        day_start = f"{iso_to_datetime(EventStart).strftime('%Y-%m-%d')}T00:00:00+05:30"
        day_end = f"{iso_to_datetime(EventEnd).strftime('%Y-%m-%d')}T23:59:59+05:30"
        events = retrive_calendar_events(email, day_start, day_end)
    
        # 🧼 Sanitize real events to ensure Attendees and NumAttendees are never empty
        for ev in events:
            if not ev.get("Attendees"):
                ev["Attendees"] = ["SELF"]
            if not ev.get("NumAttendees") or ev["NumAttendees"] == 0:
                ev["NumAttendees"] = len(ev["Attendees"])
    
        # ✅ Only append assistant-generated event if it doesn't exist
        existing = any(
            ev["StartTime"] == EventStart and ev["Summary"] == meeting_subject
            for ev in events
        )
        if not existing:
            events.append({
                "StartTime": EventStart,
                "EndTime": EventEnd,
                "NumAttendees": len(participants),
                "Attendees": participants,
                "Summary": meeting_subject
            })
    
        # 🧹 Deduplicate on (StartTime, Summary)
        unique_events = []
        seen = set()
        for ev in events:
            key = (ev["StartTime"], ev["Summary"])
            if key not in seen:
                seen.add(key)
                unique_events.append(ev)
    
        output["Attendees"].append({
            "email": email,
            "events": unique_events
        })




    return output


# ---------- FLASK API SETUP ----------
app = Flask(__name__)
received_data = []

@app.route('/receive', methods=['POST'])
def receive():
    data = request.get_json()
    new_data = your_meeting_assistant(data)
    received_data.append(data)
    return jsonify(new_data)

def run_flask():
    app.run(host='0.0.0.0', port=5008)  # <-- use a port that is free

# ---------- START FLASK SERVER ----------
Thread(target=run_flask, daemon=True).start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5008
 * Running on http://134.199.201.21:5008
Press CTRL+C to quit


In [3]:
import requests

test_json ={
    "Request_id": "6118b54f-907b-4451-8d48-dd13d76033c5",
    "Datetime": "02-07-2025T12:34:55",
    "Location": "IIT Mumbai",
    "From": "userone.amd@gmail.com",
    "Attendees": [
        {
            "email": "usertwo.amd@gmail.com"
        },
        {
            "email": "userthree.amd@gmail.com"
        }
    ],
    "Subject": "Project Status",
    "EmailContent": "Hi Team. Let's meet on next Friday at 11:00 A.M and discuss about our on-going Proje"
}



url = "http://127.0.0.1:5008/receive"  # <--- use the port you chose

response = requests.post(url, json=test_json)
print("Test API Output:\n", json.dumps(response.json(), indent=2))


Test API Output:
 {
  "Attendees": [
    {
      "email": "userthree.amd@gmail.com",
      "events": [
        {
          "Attendees": [
            "SELF"
          ],
          "EndTime": "2025-07-11T07:30:00+05:30",
          "NumAttendees": 1,
          "StartTime": "2025-07-10T16:00:00+05:30",
          "Summary": "Off Hours"
        },
        {
          "Attendees": [
            "SELF"
          ],
          "EndTime": "2025-07-12T07:30:00+05:30",
          "NumAttendees": 1,
          "StartTime": "2025-07-11T16:00:00+05:30",
          "Summary": "Off Hours"
        },
        {
          "Attendees": [
            "userthree.amd@gmail.com",
            "userone.amd@gmail.com",
            "usertwo.amd@gmail.com"
          ],
          "EndTime": "2025-07-11T11:30:00+0530",
          "NumAttendees": 3,
          "StartTime": "2025-07-11T11:00:00+0530",
          "Summary": "Project Status"
        }
      ]
    },
    {
      "email": "userone.amd@gmail.com",
      "events":